# Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import json
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/law.json') as f:
    law = json.load(f)
# with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/train.json') as f:
#     data = json.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/private_test_TASK_1.json') as f:
    question = json.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/gold_prv_24.json') as f:
    article = json.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/submit_2.json') as f:
    answer = json.load(f)
with open("/content/drive/MyDrive/Colab Notebooks/ALQAC/predict_top10_private_2024 (1).json") as f:
    predict = json.load(f)
# with open('/kaggle/input/prompt-legal/prompts.json') as f:
#     prompts = json.load(f)

In [6]:
#merge question, article and answer as a data
data = []
for i in range(len(question)):
    data.append({
        "question_id": question[i]['question_id'],
        "question_type": question[i]['question_type'],
        "text": question[i]['text'],
        # "choices": question[i]['choices'],
        "answer": answer[i]['answer'],
        "relevant_articles": article[i]['relevant_articles']
    })
    if question[i]['question_type'] == 'Trắc nghiệm':
        data[i]['choices'] = question[i]['choices']

In [62]:
for i in range(len(data)):
  #find where id predict == id data[i]
  id = data[i]['question_id']
  for j in range(len(predict)):
    if predict[j]['question_id'] == id:
      data[i]['predict_relevant_articles'] = predict[j]['predict_relevant_articles']
      break
  # data[i]['predict_relevant_articles'] = predict[i]['predict_relevant_articles']

In [46]:
print(data[0])

{'question_id': '04-DS-1', 'question_type': 'Đúng/Sai', 'text': 'Trong nguyên tắc về quản lý viên chức, việc tuyển dụng, sử dụng, quản lý, đánh giá viên chức được thực hiện trên cơ sở tiêu chuẩn chức danh nghề nghiệp, vị trí việc làm và căn cứ vào hợp đồng làm việc là đúng hay sai?', 'answer': 'Đúng', 'relevant_articles': [{'law_id': 'Luật Viên chức', 'article_id': '6', 'content': 'Các nguyên tắc quản lý viên chức\n\n1. Bảo đảm sự lãnh đạo của Đảng Cộng sản Việt Nam và sự thống nhất quản lý của Nhà nước.\n\n2. Bảo đảm quyền chủ động và đề cao trách nhiệm của người đứng đầu đơn vị sự nghiệp công lập.\n\n3. Việc tuyển dụng, sử dụng, quản lý, đánh giá viên chức được thực hiện trên cơ sở tiêu chuẩn chức danh nghề nghiệp, vị trí việc làm và căn cứ vào hợp đồng làm việc.\n\n4. Thực hiện bình đẳng giới, các chính sách ưu đãi của Nhà nước đối với viên chức là người có tài năng, người dân tộc thiểu số, người có công với cách mạng, viên chức làm việc ở miền núi, biên giới, hải đảo, vùng sâu, vùn

In [7]:
#save data to json file
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/test.json', 'w') as f:
    json.dump(data, f)

In [2]:
prompts = dict()

In [3]:
instruction = dict()
instruction['Tự luận'] = 'Chỉ cần trả lời đáp án ngắn gọn nhất có thể, không cần giải thích'
instruction['Trắc nghiệm'] = 'Chỉ có 1 đáp án đúng và chỉ cần trả lời A, B, C hoặc D, không cần giải thích'
instruction['Đúng/Sai'] = 'Chỉ cần trả lời Đúng hoặc Sai, không cần giải thích'

In [4]:
prompts['Tự luận'] = [
    "Các bài viết liên quan: {articles}\n\nCâu hỏi: {question}\n\n",
]
prompts['Trắc nghiệm'] = [
    "Các bài viết liên quan: {articles}\n\nCâu hỏi: {question}\n\nLựa chọn: {choices}\n\n",
]
prompts['Đúng/Sai'] = [
    "Các bài viết liên quan: {articles}\n\nCâu hỏi: {question}\n\n",
]


In [ ]:
# prompts['essay'] = [p + "\n\nLưu ý: Chỉ cần trả lời đáp án, không cần giải thích" for p in prompts['essay']]
# prompts['options'] = [p + "\n\nLưu ý: Chỉ có 1 đáp án đúng và chỉ cần trả lời A, B, C hoặc D, không cần giải thích" for p in prompts['options']]
# prompts['truefalse'] = [p + "\n\nLưu ý: Chỉ cần trả lời Đúng hoặc Sai, không cần giải thích" for p in prompts['truefalse']]

In [44]:
def get_relevant_articles(relevant_articles, law_data, top_k=1):
    articles_content = []
    for i in range(min(top_k-1,len(relevant_articles))):
        for law in law_data:
            if law['id'] == relevant_articles[i]['law_id']:
                for article in law['articles']:
                    if article['id'] == relevant_articles[i]['article_id']:
                        articles_content.append(article['text'])
                        break
#             raise ValueError('Article not found')
    return " ".join(articles_content)
print(get_relevant_articles(data[0]['relevant_articles'], law))

In [ ]:
def prompt_format_data(data, prompt):
    if data['question_type'] == 'Trắc nghiệm':
        return prompt.format(articles = get_relevant_articles(data[],law),
                    question = data['text'], choices = data['choices'])
    else:
        return prompt.format(articles = get_relevant_articles(data['relevant_articles'],law),
                    question = data['text'])
print(prompt_format_data(data[92], prompts["Tự luận"][-1]))

In [52]:
print(data[92]['predict_relevant_articles'][0]['content'])

Quyền và nghĩa vụ của doanh nghiệp kinh doanh dịch vụ lữ hành 1. Doanh nghiệp kinh doanh dịch vụ lữ hành nội địa có quyền và nghĩa vụ sau đây: a) Xây dựng, quảng cáo, bán và tổ chức thực hiện các dịch vụ du lịch, chương trình du lịch cho khách du lịch theo phạm vi kinh doanh quy định trong giấy phép; b) Bảo đảm duy trì các điều kiện kinh doanh dịch vụ lữ hành theo quy định tại khoản 1 Điều 31 của Luật này; công khai tên doanh nghiệp, số giấy phép kinh doanh dịch vụ lữ hành trên biển hiệu tại trụ sở, chi nhánh, văn phòng giao dịch, trong hợp đồng lữ hành, trên ấn phẩm quảng cáo và trong giao dịch điện tử; c) Thông báo về việc thay đổi người phụ trách kinh doanh dịch vụ lữ hành, gửi hồ sơ về người phụ trách kinh doanh dịch vụ lữ hành thay thế cho cơ quan nhà nước có thẩm quyền cấp giấy phép trong thời hạn 15 ngày kể từ khi thay đổi; d) Cung cấp thông tin về chương trình, dịch vụ, điểm đến du lịch cho khách du lịch; đ) Mua bảo hiểm cho khách du lịch trong thời gian thực hiện chương trình 

In [82]:
def prompt_format_data2(data, prompt, top_k=1):
    all_articles = []
    #avoid lost-in-the-middle, make 2 top first, 2 top last, 2 top after first, 2 top before last, so on...
    relevant_articles =""
    status = 0
    for i in range(top_k -1, -1, -1):
        if status % 2:
          all_articles.append(data['predict_relevant_articles'][i]['content'])
        else:
          #add to first element
          all_articles.insert(0, data['predict_relevant_articles'][i]['content'])
        status += 1
    for article in all_articles:
        relevant_articles += article + "\n"
    if data['question_type'] == 'Trắc nghiệm':
        return prompt.format(articles = relevant_articles,
                    question = data['text'], choices = data['choices'])
    else:
        return prompt.format(articles = relevant_articles,
                    question = data['text'])
print(prompt_format_data2(data[4], prompts["Đúng/Sai"][-1], 5))

Các bài viết liên quan: Chính sách về bảo vệ Tổ quốc 1. Bảo đảm cho thanh niên được giáo dục quốc phòng và an ninh, tinh thần yêu nước, chủ nghĩa anh hùng cách mạng, ý thức xây dựng và bảo vệ Tổ quốc. 2. Bảo đảm cho thanh niên thực hiện nghĩa vụ quân sự; nghĩa vụ tham gia Công an nhân dân, Dân quân tự vệ; tham gia quân nhân dự bị và xây dựng nền quốc phòng toàn dân, nền an ninh nhân dân theo quy định của pháp luật. 3. Thanh niên hoàn thành nghĩa vụ quân sự, nghĩa vụ tham gia Công an nhân dân được hưởng chính sách hỗ trợ đào tạo nghề và các chính sách ưu đãi khác theo quy định của pháp luật.
Chính sách đối với thanh niên xung phong 1. Thanh niên xung phong là lực lượng xung kích của thanh niên tham gia thực hiện các nhiệm vụ phát triển kinh tế - xã hội, giải quyết việc làm, giáo dục, đào tạo, rèn luyện thanh niên và các nhiệm vụ đột xuất, cấp bách, khó khăn, gian khổ trong xây dựng và bảo vệ Tổ quốc. 2. Nhà nước ban hành chính sách để thanh niên xung phong thực hiện các nhiệm vụ sau đây

In [110]:
import re
def process(input_string):
    # Sử dụng regular expression để thay thế các dấu câu bằng chuỗi rỗng
    no_punctuation_string = re.sub(r'[^\w\s]', '', input_string.strip().lower())
    modified_string = re.sub(r'\b0+(\d)', r'\1', no_punctuation_string)
    #remove ** from start and end
    modified_string = modified_string.strip('*')
    return modified_string
example = '**Trong 01 năm**'
print(process(example))

trong 1 năm


# LLAMA

In [12]:
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


In [117]:
import os
import time
from groq import Groq

client = Groq(
    api_key='' #get tokenn from groq
)

In [118]:
def generate_answer_groq(instruction, prompt):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": instruction,
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-70b-8192",

        max_tokens=1024,
    )

    return chat_completion.choices[0].message.content

In [16]:
answer_llama = []
num_true = 0

for i in range(0, len(data)):
    prompt = ""

    prompt = prompt_format_data(data[i], prompts[data[i]["question_type"]][-1])

    # prompt = prompt + 'Lưu ý: '+ instruction[data[i]["question_type"]]
    response = generate_answer_groq(instruction[data[i]["question_type"]], prompt)
    # print(data[i]['question_id'], ':', response)
    if process(response) == process(data[i]["answer"]):
        num_true += 1
    else:
        print('False in data: ', data[i]["question_id"], 'with answer: ', response, 'and true: ', data[i]["answer"])
    answer_llama.append({"question_id": data[i]['question_id'],
                        "answer": response})
print('accuracy: ', num_true/len(data) * 100)


False in data:  04-DS-6 with answer:  ĐÚNG and true:  Sai
False in data:  04-DS-13 with answer:  Sai and true:  Đúng
False in data:  04-DS-19 with answer:  ĐÚNG and true:  Sai
False in data:  04-TN-9 with answer:  B and true:  C
False in data:  04-TN-10 with answer:  B and true:  D
False in data:  04-TN-27 with answer:  D and true:  C
False in data:  04-TL-1 with answer:  03 ngày. and true:  03 ngày làm việc.
False in data:  04-TL-3 with answer:  6 tháng. and true:  Trước ngày 30/11/2023.
False in data:  04-TL-4 with answer:  Giá trị. and true:  Chia theo giá trị.
False in data:  04-TL-5 with answer:  300 and true:  300 ngày.
False in data:  04-TL-6 with answer:  Chương trình, sách giáo khoa, giáo trình, tài liệu. and true:  Chương trình, sách giáo khoa, giáo trình, tài liệu giáo dục thường xuyên.
False in data:  04-TL-8 with answer:  Có, trong thời hạn 30 ngày. and true:  Công ty A có quyền yêu cầu sửa chữa lỗi tính toán sai trong phán quyết. Hội đồng trọng tài cần phải đáp ứng yêu cầ

## top_k = 1

In [64]:
answer_llama2 = []
num_true = 0

for i in range(0, len(data)):
    prompt = ""

    prompt = prompt_format_data2(data[i], prompts[data[i]["question_type"]][-1])

    # prompt = prompt + 'Lưu ý: '+ instruction[data[i]["question_type"]]
    response = generate_answer_groq(instruction[data[i]["question_type"]], prompt)
    # print(data[i]['question_id'], ':', response)
    if process(response) == process(data[i]["answer"]):
        num_true += 1
    else:
        print('False in data: ', data[i]["question_id"], 'with answer: ', response, 'and true: ', data[i]["answer"])
    answer_llama2.append({"question_id": data[i]['question_id'],
                        "answer": response})
print('accuracy: ', num_true/len(data) * 100)


False in data:  04-DS-9 with answer:  ĐÚNG and true:  Sai
False in data:  04-DS-13 with answer:  Sai and true:  Đúng
False in data:  04-DS-25 with answer:  Sai and true:  Đúng
False in data:  04-TN-7 with answer:  D and true:  A
False in data:  04-TN-9 with answer:  D and true:  C
False in data:  04-TN-10 with answer:  B and true:  D
False in data:  04-TN-41 with answer:  C and true:  B
False in data:  04-TN-42 with answer:  B and true:  A
False in data:  04-TL-1 with answer:  03 ngày. and true:  03 ngày làm việc.
False in data:  04-TL-3 with answer:  01/12/2023. and true:  Trước ngày 30/11/2023.
False in data:  04-TL-4 with answer:  Giá trị. and true:  Chia theo giá trị.
False in data:  04-TL-5 with answer:  300 and true:  300 ngày.
False in data:  04-TL-6 with answer:  Chương trình, sách giáo khoa, giáo trình, tài liệu. and true:  Chương trình, sách giáo khoa, giáo trình, tài liệu giáo dục thường xuyên.
False in data:  04-TL-8 with answer:  Không. Within 30 days. and true:  Công ty A

In [66]:
print(len(answer_llama2))
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/llama_v2.json', 'w', encoding='utf-8') as f:
    json.dump(answer_llama2, f, ensure_ascii=False, indent=4)

100


# top_k = 5

In [ ]:
answer_llama3 = []
num_true = 0

for i in range(0, len(data)):
    prompt = ""

    prompt = prompt_format_data2(data[i], prompts[data[i]["question_type"]][-1], top_k = 5)

    # prompt = prompt + 'Lưu ý: '+ instruction[data[i]["question_type"]]
    response = generate_answer_groq(instruction[data[i]["question_type"]], prompt)
    # print(data[i]['question_id'], ':', response)
    if process(response) == process(data[i]["answer"]):
        num_true += 1
    else:
        print('False in data: ', data[i]["question_id"], 'with answer: ', response, 'and true: ', data[i]["answer"])
    answer_llama3.append({"question_id": data[i]['question_id'],
                        "answer": response})
    time.sleep(3)
print('accuracy: ', num_true/len(data) * 100)


In [95]:
print(len(answer_llama3))
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/llama_v3.json', 'w', encoding='utf-8') as f:
    json.dump(answer_llama3, f, ensure_ascii=False, indent=4)

100


## top_k = 10

In [ ]:
answer_llama4 = []
num_true = 0

for i in range(13, len(data)):
    prompt = ""

    prompt = prompt_format_data2(data[i], prompts[data[i]["question_type"]][-1], top_k = 10)

    # prompt = prompt + 'Lưu ý: '+ instruction[data[i]["question_type"]]
    response = generate_answer_groq(instruction[data[i]["question_type"]], prompt)
    # print(data[i]['question_id'], ':', response)
    if process(response) == process(data[i]["answer"]):
        num_true += 1
    else:
        print('False in data: ', data[i]["question_id"], 'with answer: ', response, 'and true: ', data[i]["answer"])
    answer_llama4.append({"question_id": data[i]['question_id'],
                        "answer": response})
    time.sleep(3)
print('accuracy: ', num_true/len(data) * 100)


# Gemini

In [20]:
pip install -q -U google-generativeai

In [92]:
import google.generativeai as genai
import os
import time
genai.configure(api_key="") #get token from google

model = genai.GenerativeModel('gemini-1.5-flash')

In [93]:
config = genai.GenerationConfig(
    max_output_tokens=1024,
)

In [94]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold
safety_settings={

        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,

        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,

        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE

    }

In [26]:
answer_gemini = []
num_true = 0
for i in range(0, len(data)):
    prompt = prompt_format_data(data[i], prompts[data[i]["question_type"]][-1])
    prompt = prompt + 'Lưu ý: '+ instruction[data[i]["question_type"]]
    response = model.generate_content(prompt, generation_config=config, safety_settings=safety_settings).text
    if process(response) == process(data[i]["answer"]):
        num_true += 1
    else:
        print('False in data: ', data[i]["question_id"], 'with answer: ', response, 'and true: ', data[i]["answer"])
    answer_gemini.append({"question_id": data[i]['question_id'],
                        "answer": response})
    time.sleep(5)
print('accuracy: ', num_true/len(data) * 100)

False in data:  04-DS-16 with answer:  Đúng. 
 and true:  Sai
False in data:  04-DS-48 with answer:  Sai 
 and true:  Đúng
False in data:  04-TN-8 with answer:  Đáp án **C**. 

Theo phần 2 của bài viết, những người đang bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở chữa bệnh không được đăng ký dự tuyển viên chức. 
 and true:  C
False in data:  04-TN-9 with answer:  D 
 and true:  C
False in data:  04-TN-10 with answer:  Đáp án: **D** 
 and true:  D
False in data:  04-TN-17 with answer:  Đáp án: **B** 
 and true:  B
False in data:  04-TN-24 with answer:  Đáp án: **A** 
 and true:  A
False in data:  04-TN-27 with answer:  Đáp án **D**. 

Theo quy định, giữa năm là ngày cuối cùng của tháng 6, không phải ngày 30 tháng 6. 
 and true:  C
False in data:  04-TN-30 with answer:  Đáp án **B**.

* Theo quy định, khi thời hạn tính bằng ngày, ngày đầu tiên của thời hạn không được tính mà tính từ ngày tiếp theo liền kề ngày được xác định.
* Do đó, thời hạn của thỏa thuận giữa A và B bắt đầu từ

In [27]:
print(len(answer_gemini))
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/gemini_v1.json', 'w', encoding='utf-8') as f:
    json.dump(answer_gemini, f, ensure_ascii=False, indent=4)

100


## top_k=1

In [67]:
answer_gemini2 = []
num_true = 0
for i in range(0, len(data)):
    prompt = prompt_format_data2(data[i], prompts[data[i]["question_type"]][-1])
    prompt = prompt + 'Lưu ý: '+ instruction[data[i]["question_type"]]
    response = model.generate_content(prompt, generation_config=config, safety_settings=safety_settings).text
    if process(response) == process(data[i]["answer"]):
        num_true += 1
    else:
        print('False in data: ', data[i]["question_id"], 'with answer: ', response, 'and true: ', data[i]["answer"])
    answer_gemini2.append({"question_id": data[i]['question_id'],
                        "answer": response})
    time.sleep(5)
print('accuracy: ', num_true/len(data) * 100)

False in data:  04-TN-7 with answer:  **D** 
 and true:  A
False in data:  04-TN-10 with answer:  D. 3/4/2025 
 and true:  D
False in data:  04-TN-15 with answer:  B. 
 and true:  C
False in data:  04-TN-24 with answer:  D 
 and true:  A
False in data:  04-TN-27 with answer:  Đáp án: **B** 
 and true:  C
False in data:  04-TN-30 with answer:  D 
 and true:  B
False in data:  04-TN-34 with answer:  Đáp án **C**. 

Các đáp án còn lại sai:

* **A:** Thời gian lưu trữ bị giới hạn bởi pháp luật.
* **B:** Các loại rác thải phải được phân loại và lưu trữ riêng biệt.
* **D:** Rò rỉ bụi, chất thải dạng lỏng ra môi trường là vi phạm và phải được ngăn chặn. 
 and true:  C
False in data:  04-TN-37 with answer:  Đáp án: **C** 
 and true:  C
False in data:  04-TN-41 with answer:  D. 
 and true:  B
False in data:  04-TL-1 with answer:  03 ngày. 
 and true:  03 ngày làm việc.
False in data:  04-TL-3 with answer:  29/11/2023 
 and true:  Trước ngày 30/11/2023.
False in data:  04-TL-6 with answer:  Chươ

In [68]:
print(len(answer_gemini2))
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/gemini_v2.json', 'w', encoding='utf-8') as f:
    json.dump(answer_gemini2, f, ensure_ascii=False, indent=4)

100


#top_k = 5

In [96]:
answer_gemini3 = []
num_true = 0
for i in range(0, len(data)):
    prompt = prompt_format_data2(data[i], prompts[data[i]["question_type"]][-1], top_k = 5)
    prompt = prompt + 'Lưu ý: '+ instruction[data[i]["question_type"]]
    response = model.generate_content(prompt, generation_config=config, safety_settings=safety_settings).text
    if process(response) == process(data[i]["answer"]):
        num_true += 1
    else:
        print('False in data: ', data[i]["question_id"], 'with answer: ', response, 'and true: ', data[i]["answer"])
    answer_gemini3.append({"question_id": data[i]['question_id'],
                        "answer": response})
    time.sleep(5)
print('accuracy: ', num_true/len(data) * 100)

False in data:  04-DS-16 with answer:  Đúng. 
 and true:  Sai
False in data:  04-TN-6 with answer:  Đáp án: **B** 
 and true:  B
False in data:  04-TN-7 with answer:  Đáp án đúng là **A**. 

Theo đoạn văn về Chế độ thôi việc, viên chức không được hưởng trợ cấp thôi việc nếu bị buộc thôi việc. 
 and true:  A
False in data:  04-TN-8 with answer:  Đáp án **C**. 

Theo nội dung bài viết, những người đang bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở chữa bệnh không được đăng ký dự tuyển viên chức.
 and true:  C
False in data:  04-TN-9 with answer:  D. 
 and true:  C
False in data:  04-TN-10 with answer:  Đáp án: **D** 
 and true:  D
False in data:  04-TN-24 with answer:  Đáp án: **A** 
 and true:  A
False in data:  04-TN-27 with answer:  B. 
 and true:  C
False in data:  04-TN-28 with answer:  Đáp án: **C** 
 and true:  D
False in data:  04-TN-30 with answer:  D 
 and true:  B
False in data:  04-TN-31 with answer:  Đáp án: **C** 
 and true:  C
False in data:  04-TN-36 with answer:  Đ

In [108]:
config2 = genai.GenerationConfig(
    max_output_tokens=1024, temperature = 0
)

In [109]:
i=53
prompt = prompt_format_data2(data[i], prompts[data[i]["question_type"]][-1], top_k = 5)
prompt = prompt + 'Lưu ý: '+ instruction[data[i]["question_type"]]
response = model.generate_content(prompt, generation_config=config, safety_settings=safety_settings).text
print(prompt, response)

Các bài viết liên quan: Áp dụng điều ước quốc tế về quyền trẻ em đối với thanh niên từ đủ 16 tuổi đến dưới 18 tuổi Nhà nước áp dụng điều ước quốc tế về quyền trẻ em mà nước Cộng hòa xã hội chủ nghĩa Việt Nam là thành viên đối với thanh niên từ đủ 16 tuổi đến dưới 18 tuổi phù hợp với điều kiện của Việt Nam.
Thanh niên Thanh niên là công dân Việt Nam từ đủ 16 tuổi đến 30 tuổi.
Bảo vệ trẻ em trên không gian mạng 1. Trẻ em có quyền được bảo vệ, tiếp cận thông tin, tham gia hoạt động xã hội, vui chơi, giải trí, giữ bí mật cá nhân, đời sống riêng tư và các quyền khác khi tham gia trên không gian mạng. 2. Chủ quản hệ thống thông tin, doanh nghiệp cung cấp dịch vụ trên mạng viễn thông, mạng Internet, các dịch vụ gia tăng trên không gian mạng có trách nhiệm kiểm soát nội dung thông tin trên hệ thống thông tin hoặc trên dịch vụ do doanh nghiệp cung cấp để không gây nguy hại cho trẻ em, xâm phạm đến trẻ em, quyền trẻ em; ngăn chặn việc chia sẻ và xóa bỏ thông tin có nội dung gây nguy hại cho trẻ 

In [97]:
print(len(answer_gemini3))
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/gemini_v3.json', 'w', encoding='utf-8') as f:
    json.dump(answer_gemini3, f, ensure_ascii=False, indent=4)

100


## top_k = 10

In [111]:
answer_gemini4 = []
num_true = 0
for i in range(0, len(data)):
    prompt = prompt_format_data2(data[i], prompts[data[i]["question_type"]][-1], top_k = 10)
    prompt = prompt + 'Lưu ý: '+ instruction[data[i]["question_type"]]
    response = model.generate_content(prompt, generation_config=config, safety_settings=safety_settings).text
    if process(response) == process(data[i]["answer"]):
        num_true += 1
    else:
        print('False in data: ', data[i]["question_id"], 'with answer: ', response, 'and true: ', data[i]["answer"])
    answer_gemini4.append({"question_id": data[i]['question_id'],
                        "answer": response})
    time.sleep(5)
print('accuracy: ', num_true/len(data) * 100)

False in data:  04-DS-16 with answer:  Đúng. 
 and true:  Sai
False in data:  04-DS-31 with answer:  Sai. 
 and true:  Đúng
False in data:  04-DS-44 with answer:  Đúng 
 and true:  Sai
False in data:  04-TN-9 with answer:  D 
 and true:  C
False in data:  04-TN-10 with answer:  D. 3/4/2025 
 and true:  D
False in data:  04-TN-12 with answer:  Đáp án đúng là **B**. 

Theo đoạn văn, trong trường hợp cả cha và mẹ đều không đủ điều kiện trực tiếp nuôi con thì Tòa án quyết định giao con cho **người giám hộ** theo quy định của Bộ luật dân sự. 
 and true:  B
False in data:  04-TN-17 with answer:  B. 06 tháng 
 and true:  B
False in data:  04-TN-20 with answer:  Đáp án: **B** 
 and true:  B
False in data:  04-TN-24 with answer:  Đáp án: **A** 
 and true:  A
False in data:  04-TN-27 with answer:  Đáp án: **B** 
 and true:  C
False in data:  04-TN-28 with answer:  Đáp án **D**. 

Điều 125, 126, 127, 128 và 129 đều liên quan đến các trường hợp cụ thể có thời hiệu yêu cầu Tòa án tuyên bố giao dịch

In [112]:
print(len(answer_gemini4))
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/gemini_v4.json', 'w', encoding='utf-8') as f:
    json.dump(answer_gemini4, f, ensure_ascii=False, indent=4)

100


# GPT (đoạn này chưa code vì ko có tiền mua token)

In [ ]:
!pip install openai

In [ ]:
OPENAI_API_KEY = '' #get token from openai
from openai import OpenAI

client = OpenAI(
  api_key=OPENAI_API_KEY,
)

In [ ]:
def completion_gpt(prompt):
    completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": prompt}
        ], temperature=0
    )
    return str(completion.choices[0].message.content)

In [ ]:
answer_gpt = []
for i in range(0, len(data)):
    prompt = ""
    if data[i]["question_type"] == "Đúng/Sai":
        prompt = prompt_format_data(data[i], prompts["truefalse"][-1])
    elif data[i]["question_type"] == "Trắc nghiệm":
        prompt = prompt_format_data(data[i], prompts["options"][-1])
    else:
        prompt = prompt_format_data(data[i], prompts["essay"][-1])
    response = completion_gpt(prompt)
    print(data[i]['question_id'], ':', response)
    answer_gpt.append({"question_id": data[i]['question_id'],
                        "answer": response})


# Qwen

In [28]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key='')

answer_qwen = []
num_true = 0

for i in range(0, len(data)):
    prompt = ""
    prompt = prompt_format_data(data[i], prompts[data[i]["question_type"]][-1])
    responses = client.chat_completion(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=[{"role": "system", "content": instruction[data[i]["question_type"]]},
                  {"role": "user", "content": prompt}],
        stream=True,
        max_tokens=10000,
    )
    response = ""

    for message in responses:
      response += message.choices[0].delta.content

    # print(data[i]['question_id'], ':', response)
    if process(response) == process(data[i]["answer"]):
          num_true += 1
    else:
          print('False in data: ', data[i]["question_id"], 'with answer: ', response, 'and true: ', data[i]["answer"])
    answer_qwen.append({"question_id": data[i]['question_id'],
                        "answer": response})

print('accuracy: ', num_true/len(data) * 100)

  #   for message in client.chat_completion(
  #   model="Qwen/Qwen2.5-72B-Instruct",
  #   messages=[{"role": "user", "content": prompt_format_data(data[3], prompts["truefalse"][-1])}],
  #   stream=True,
  #   max_tokens=500,
  # ):
  #     print(message.choices[0].delta.content, end="")

False in data:  04-DS-6 with answer:  Đúng and true:  Sai
False in data:  04-DS-19 with answer:  Đúng and true:  Sai
False in data:  04-DS-45 with answer:  Sai. Du lịch sinh thái mới chính là loại hình du lịch dựa vào thiên nhiên, gắn với bản sắc văn hoá địa phương, có sự tham gia của cộng đồng dân cư, kết hợp giáo dục về bảo vệ môi trường. Du lịch cộng đồng là loại hình du lịch được phát triển trên cơ sở các giá trị văn hóa của cộng đồng, do cộng đồng dân cư quản lý, tổ chức khai thác và hưởng lợi. and true:  Sai
False in data:  04-TN-9 with answer:  D and true:  C
False in data:  04-TN-10 with answer:  A and true:  D
False in data:  04-TN-27 with answer:  D and true:  C
False in data:  04-TL-3 with answer:  30/11/2023 and true:  Trước ngày 30/11/2023.
accuracy:  93.0


In [29]:
print(len(answer_qwen))
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/qwen_v1.json', 'w', encoding='utf-8') as f:
    json.dump(answer_qwen, f, ensure_ascii=False, indent=4)

100


## top_k = 1

In [69]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key="")

answer_qwen2 = []
num_true = 0

for i in range(0, len(data)):
    prompt = ""
    prompt = prompt_format_data2(data[i], prompts[data[i]["question_type"]][-1])
    responses = client.chat_completion(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=[{"role": "system", "content": instruction[data[i]["question_type"]]},
                  {"role": "user", "content": prompt}],
        stream=True,
        max_tokens=10000,
    )
    response = ""

    for message in responses:
      response += message.choices[0].delta.content

    # print(data[i]['question_id'], ':', response)
    if process(response) == process(data[i]["answer"]):
          num_true += 1
    else:
          print('False in data: ', data[i]["question_id"], 'with answer: ', response, 'and true: ', data[i]["answer"])
    answer_qwen2.append({"question_id": data[i]['question_id'],
                        "answer": response})

print('accuracy: ', num_true/len(data) * 100)

  #   for message in client.chat_completion(
  #   model="Qwen/Qwen2.5-72B-Instruct",
  #   messages=[{"role": "user", "content": prompt_format_data(data[3], prompts["truefalse"][-1])}],
  #   stream=True,
  #   max_tokens=500,
  # ):
  #     print(message.choices[0].delta.content, end="")

False in data:  04-DS-9 with answer:  Đúng and true:  Sai
False in data:  04-DS-13 with answer:  Sai and true:  Đúng
False in data:  04-DS-25 with answer:  Sai and true:  Đúng
False in data:  04-DS-44 with answer:  Đúng and true:  Sai
False in data:  04-DS-45 with answer:  Đúng and true:  Sai
False in data:  04-TN-9 with answer:  D and true:  C
False in data:  04-TN-10 with answer:  A and true:  D
False in data:  04-TN-27 with answer:  D and true:  C
False in data:  04-TN-41 with answer:  A and true:  B
False in data:  04-TL-2 with answer:  2 năm liên tục and true:  02 năm liên tiếp.
False in data:  04-TL-3 with answer:  30/11/2023 and true:  Trước ngày 30/11/2023.
False in data:  04-TL-8 with answer:  Công ty A có quyền yêu cầu sửa chữa lỗi tính toán sai trong phán quyết theo quy định của Điều 51 Luật Trọng tài thương mại. Hội đồng trọng tài cần phải đáp ứng yêu cầu này trong thời hạn 30 ngày, kể từ ngày nhận được yêu cầu sửa chữa. and true:  Công ty A có quyền yêu cầu sửa chữa lỗi tí

In [70]:
print(len(answer_qwen2))
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/qwen_v2.json', 'w', encoding='utf-8') as f:
    json.dump(answer_qwen2, f, ensure_ascii=False, indent=4)

100


## top_k = 5

In [98]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key="")

answer_qwen3 = []
num_true = 0

for i in range(0, len(data)):
    prompt = ""
    prompt = prompt_format_data2(data[i], prompts[data[i]["question_type"]][-1], top_k = 5)
    responses = client.chat_completion(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=[{"role": "system", "content": instruction[data[i]["question_type"]]},
                  {"role": "user", "content": prompt}],
        stream=True,
        max_tokens=10000,
    )
    response = ""

    for message in responses:
      response += message.choices[0].delta.content

    # print(data[i]['question_id'], ':', response)
    if process(response) == process(data[i]["answer"]):
          num_true += 1
    else:
          print('False in data: ', data[i]["question_id"], 'with answer: ', response, 'and true: ', data[i]["answer"])
    answer_qwen3.append({"question_id": data[i]['question_id'],
                        "answer": response})

print('accuracy: ', num_true/len(data) * 100)

  #   for message in client.chat_completion(
  #   model="Qwen/Qwen2.5-72B-Instruct",
  #   messages=[{"role": "user", "content": prompt_format_data(data[3], prompts["truefalse"][-1])}],
  #   stream=True,
  #   max_tokens=500,
  # ):
  #     print(message.choices[0].delta.content, end="")

False in data:  04-DS-24 with answer:  Sai. Cộng đồng nơi người nghiện ma túy cư trú có trách nhiệm động viên, giúp đỡ người nghiện ma túy và phối hợp với cơ quan, tổ chức, đơn vị có liên quan hỗ trợ người nghiện ma túy trong quá trình cai nghiện ma túy, quản lý sau cai nghiện ma túy và hòa nhập cộng đồng, không phải xa lánh họ. and true:  Sai
False in data:  04-DS-44 with answer:  Đúng and true:  Sai
False in data:  04-DS-45 with answer:  Đúng and true:  Sai
False in data:  04-TN-9 with answer:  D and true:  C
False in data:  04-TN-10 with answer:  A and true:  D
False in data:  04-TN-27 with answer:  D and true:  C
False in data:  04-TL-3 with answer:  30/11/2023 and true:  Trước ngày 30/11/2023.
accuracy:  93.0


In [99]:
print(len(answer_qwen3))
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/qwen_v3.json', 'w', encoding='utf-8') as f:
    json.dump(answer_qwen3, f, ensure_ascii=False, indent=4)

100


## top_k = 10


In [114]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key="")

answer_qwen4 = []
num_true = 0

for i in range(0, len(data)):
    prompt = ""
    prompt = prompt_format_data2(data[i], prompts[data[i]["question_type"]][-1], top_k = 10)
    responses = client.chat_completion(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=[{"role": "system", "content": instruction[data[i]["question_type"]]},
                  {"role": "user", "content": prompt}],
        stream=True,
        max_tokens=10000,
    )
    response = ""

    for message in responses:
      response += message.choices[0].delta.content

    # print(data[i]['question_id'], ':', response)
    if process(response) == process(data[i]["answer"]):
          num_true += 1
    else:
          print('False in data: ', data[i]["question_id"], 'with answer: ', response, 'and true: ', data[i]["answer"])
    answer_qwen4.append({"question_id": data[i]['question_id'],
                        "answer": response})

print('accuracy: ', num_true/len(data) * 100)

  #   for message in client.chat_completion(
  #   model="Qwen/Qwen2.5-72B-Instruct",
  #   messages=[{"role": "user", "content": prompt_format_data(data[3], prompts["truefalse"][-1])}],
  #   stream=True,
  #   max_tokens=500,
  # ):
  #     print(message.choices[0].delta.content, end="")

False in data:  04-DS-9 with answer:  Đúng and true:  Sai
False in data:  04-DS-44 with answer:  Đúng and true:  Sai
False in data:  04-DS-45 with answer:  Sai. Du lịch cộng đồng được định nghĩa là loại hình du lịch được phát triển trên cơ sở các giá trị văn hóa của cộng đồng, do cộng đồng dân cư quản lý, tổ chức khai thác và hưởng lợi. Trong khi du lịch sinh thái mới là loại hình du lịch dựa vào thiên nhiên, gắn với bản sắc văn hoá địa phương, có sự tham gia của cộng đồng dân cư, kết hợp giáo dục về bảo vệ môi trường. and true:  Sai
False in data:  04-TN-9 with answer:  D and true:  C
False in data:  04-TN-27 with answer:  D and true:  C
False in data:  04-TN-30 with answer:  A and true:  B
False in data:  04-TN-41 with answer:  A and true:  B
False in data:  04-TL-3 with answer:  30/11/2023 and true:  Trước ngày 30/11/2023.
accuracy:  92.0


In [115]:
print(len(answer_qwen4))
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/qwen_v4.json', 'w', encoding='utf-8') as f:
    json.dump(answer_qwen4, f, ensure_ascii=False, indent=4)

100


# Mixtral

In [30]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key="")

answer_mixtral = []
num_true = 0

for i in range(0, len(data)):
    prompt = ""
    prompt = prompt_format_data(data[i], prompts[data[i]["question_type"]][-1])
    responses = client.chat_completion(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        messages=[{"role": "system", "content": "Trả lời bằng tiếng Việt. " + instruction[data[i]["question_type"]]},
                  {"role": "user", "content": prompt}],
        stream=True,
        max_tokens=10000,

    )
    response = ""

    for message in responses:
      response += message.choices[0].delta.content

    # print(data[i]['question_id'], ':', response)
    response = process(response)
    if response.startswith('câu trả lời'):
        response = response.split('câu trả lời')[1].strip()
    if response.startswith('đúng'):
            response = 'Đúng'
    elif response.startswith('sai'):
            response = 'Sai'
    elif data[i]["question_type"] == 'Trắc nghiệm':
            if response.startswith('a'):
                response = 'A'
            elif response.startswith('b'):
                response = 'B'
            elif response.startswith('c'):
                response = 'C'
            elif response.startswith('d'):
                response = 'D'
    if process(response) == process(data[i]["answer"]):
          num_true += 1
    else:
          print('False in data: ', data[i]["question_id"], 'with answer: ', response, 'and true: ', data[i]["answer"])
    answer_mixtral.append({"question_id": data[i]['question_id'],
                        "answer": response})

print('accuracy: ', num_true/len(data) * 100)


False in data:  04-DS-6 with answer:  Đúng and true:  Sai
False in data:  04-DS-9 with answer:  Đúng and true:  Sai
False in data:  04-DS-11 with answer:  Đúng and true:  Sai
False in data:  04-DS-31 with answer:  Sai and true:  Đúng
False in data:  04-DS-35 with answer:  Sai and true:  Đúng
False in data:  04-DS-43 with answer:  Sai and true:  Đúng
False in data:  04-TN-3 with answer:  B and true:  D
False in data:  04-TN-6 with answer:  the correct answer is b đủ 16 tuổi đến dưới 18 tuổi from 16 years old to under 18 years old according to the given statement vietnam applies the international conventions on childrens rights to individuals from 16 to under 18 years old and true:  B
False in data:  04-TN-9 with answer:  the correct answer is d 1352030

people in vietnam become thanh niên youth when they turn 16 years old so in the case of person m they will be considered a youth when they turn 16 which falls on may 13 2030 and true:  C
False in data:  04-TN-10 with answer:  C and true:

In [31]:
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/mixtral_v1.json', 'w', encoding='utf-8') as f:
    json.dump(answer_mixtral, f, ensure_ascii=False, indent=4)

# Post processing

In [ ]:
# get answer from answer format

def get_answer(model_answer_preprocess):
    assert len(model_answer_preprocess) == len(data)
    for i in range(len(data)):
        type_question = data[i]['question_type']
        if type_question == 'Đúng/Sai':
            if model_answer_preprocess[i]["answer"].lower().__contains__('đúng'):
                # data[i]['answer'] = "Đúng"
                model_answer_preprocess[i]["answer"] = 'Đúng'
            elif model_answer_preprocess[i]["answer"].lower().__contains__('sai'):
                # data[i]['answer'] = "Sai"
                model_answer_preprocess[i]["answer"] = 'Sai'
            else:
                print(model_answer_preprocess[i]["answer"])
        elif type_question == 'Trắc nghiệm':
            if model_answer_preprocess[i]["answer"].lower()[0] == 'a':
                # data[i]['answer'] = 'A'
                model_answer_preprocess[i]["answer"] = 'A'
            elif model_answer_preprocess[i]["answer"].lower()[0] == 'b':
                # data[i]['answer'] = 'B'
                model_answer_preprocess[i]["answer"] = 'B'
            elif model_answer_preprocess[i]["answer"].lower()[0] == 'c':
                # data[i]['answer'] = 'C'
                model_answer_preprocess[i]["answer"] = 'C'
            elif model_answer_preprocess[i]["answer"].lower()[0] == 'd':
                # data[i]['answer'] = 'D'
                model_answer_preprocess[i]["answer"] = 'D'
            else:
                print(model_answer_preprocess[i]["answer"])
                raise ValueError('Answer not found')

        elif type_question == 'Tự luận':
            # data[i]['answer'] = model_answer_preprocess[i]
            model_answer_preprocess[i]["answer"] = model_answer_preprocess[i]["answer"]

    return model_answer_preprocess

answer_llama = get_answer(answer_llama)
answer_gemini = get_answer(answer_gemini)
answer_gpt = get_answer(answer_gpt)

In [ ]:
answer_gpt

# Save to array

In [65]:
print(len(answer_llama2))
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/llama_v1.json', 'w', encoding='utf-8') as f:
    json.dump(answer_llama, f, ensure_ascii=False, indent=4)

100


In [ ]:
print(len(answer_gpt))
with open('gpt_v1.json', 'w', encoding='utf-8') as f:
    json.dump(answer_gpt, f, ensure_ascii=False, indent=4)

# Ensemble

In [101]:
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/llama_v3.json') as f:
    model_1 = json.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/gemini_v3.json') as f:
    model_2 = json.load(f)
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/qwen_v3.json') as f:
    model_3 = json.load(f)
for i in range(len(model_1)):
    model_1[i]["answer"] = process(model_1[i]["answer"])
    model_2[i]["answer"] = process(model_2[i]["answer"])
    model_3[i]["answer"] = process(model_3[i]["answer"])
# If you want to ensemble more result files, the code to read additional files is here.

In [102]:
list_answers = [model_1, model_2, model_3]

In [103]:
print(model_1[54])

{'question_id': '04-TN-7', 'answer': 'a'}


In [104]:
new_answers = []
for i in range(len(list_answers[0])):
    if "TL" in list_answers[0][i]["question_id"]:
        new_answers.append({"question_id": list_answers[0][i]["question_id"],
                          "answer": list_answers[2][i]["answer"]})
    elif "DS" in list_answers[0][i]["question_id"]:
        answer = [list_answers[j][i]["answer"] for j in range(len(list_answers))]
        num_y = answer.count('đúng')
        num_n = answer.count('sai')
        if num_y > num_n:
            new_answers.append({"question_id": list_answers[0][i]["question_id"],
                          "answer": 'đúng'})
        else:
            new_answers.append({"question_id": list_answers[0][i]["question_id"],
                          "answer": 'sai'})
    else:
        answer = [list_answers[j][i]["answer"] for j in range(len(list_answers))]
        num_A = answer.count('a')
        num_B = answer.count('b')
        num_C = answer.count('c')
        num_D = answer.count('d')
        mx = max([num_A, num_B, num_C, num_D])
        if mx == num_A:
            new_answers.append({"question_id": list_answers[0][i]["question_id"],
                          "answer": 'A'})
        elif mx == num_B:
            new_answers.append({"question_id": list_answers[0][i]["question_id"],
                          "answer": 'B'})
        elif mx == num_C:
            new_answers.append({"question_id": list_answers[0][i]["question_id"],
                          "answer": 'C'})
        elif mx == num_D:
            new_answers.append({"question_id": list_answers[0][i]["question_id"],
                          "answer": 'D'})
len(list_answers[0]), len(new_answers)

(100, 100)

In [105]:
with open('/content/drive/MyDrive/Colab Notebooks/ALQAC/ensemble3.json', 'w', encoding='utf-8') as f:
    json.dump(new_answers, f, ensure_ascii=False, indent=4)

In [80]:
num_true=0
for i in range(len(new_answers)):
    response = new_answers[i]["answer"]
    true_response = data[i]["answer"]
    if process(response) != process(true_response):
        print(new_answers[i]['question_id'], response, true_response)
    else:
      num_true += 1
print('accuracy: ', num_true)

04-DS-9 đúng Sai
04-DS-13 sai Đúng
04-DS-25 sai Đúng
04-TN-7 D A
04-TN-9 D C
04-TN-10 A D
04-TN-41 A B
04-TL-2 2 năm liên tục 02 năm liên tiếp.
04-TL-3 30112023 Trước ngày 30/11/2023.
04-TL-8 công ty a có quyền yêu cầu sửa chữa lỗi tính toán sai trong phán quyết theo quy định của điều 51 luật trọng tài thương mại hội đồng trọng tài cần phải đáp ứng yêu cầu này trong thời hạn 30 ngày kể từ ngày nhận được yêu cầu sửa chữa Công ty A có quyền yêu cầu sửa chữa lỗi tính toán sai trong phán quyết. Hội đồng trọng tài cần phải đáp ứng yêu cầu này trong thời hạn 30 ngày, kể từ ngày nhận được yêu cầu.
accuracy:  90


## top_k=5

In [106]:
num_true=0
for i in range(len(new_answers)):
    response = new_answers[i]["answer"]
    true_response = data[i]["answer"]
    if process(response) != process(true_response):
        print(new_answers[i]['question_id'], response, true_response)
    else:
      num_true += 1
print('accuracy: ', num_true)

04-DS-44 đúng Sai
04-TN-9 D C
04-TN-10 A D
04-TN-27 D C
04-TL-3 30112023 Trước ngày 30/11/2023.
accuracy:  95
